# Customviz - A concept notebook for creating custom configurations
This notebook provides a whiteboard workspace for developing/hacking the infrastructure required for aiding the development of custom configurations. This builds off of a preliminary investigation by Brett earlier on. For clarity, I've included Brett's notebook cells below:

## Hacking a generic custom viewer

Brett Morris

<div class="alert alert-warning">
    <strong>Disclaimer</strong>: This hack exploits some flexibility in jdaviz that was not built for this purpose. Be careful!
</div>

The following changes must be made to `jdaviz/app.py:Application._on_new_viewer` [at roughly this line in the source code](https://github.com/spacetelescope/jdaviz/blob/f38a643d375e08b01241345c672e760407e1a4a5/jdaviz/app.py#L1680) before running the cells below: 

```diff
        elif len(self._viewer_store):
            # The plugin would only not exist for instances of Imviz where the user has
            # intentionally removed the Links Control plugin, but in that case we will
            # adopt "linked_by_wcs" from the first (assuming all are the same)
            # NOTE: deleting the default viewer is forbidden both by API and UI, but if
            # for some reason that was the case here, linked_by_wcs will default to False
-            viewer.state.linked_by_wcs = list(self._viewer_store.values())[0].state.linked_by_wcs
+            viewer.state.linked_by_wcs = getattr(
+                list(self._viewer_store.values())[0].state, 'linked_by_wcs', False
+            )
```

First let's generate some heterogeneous data for loading into our custom, flexible viewer. There will be both spectra *and* images to load (but imagine that we don't want Cubeviz or Mosviz).

The resulting `heterogeneous_data` dictionary contains within it a few dictionaries. Each inner dictionary contains `'data'` and `'viewer'` keys, which specify the data to be loaded and the viewer that knows how to handle that data.

Now let's do something reckless...

## A "ConfigHelper Helper"
Currently, our configurations are defined in YAML files that are "almost" hardcoded into the code. But all this is doing in the backend is reading the file, constructing a dictionary from the contents, and passing it down the line. So, if we had a helper to help us make our own dictionary, we should be able to pass that in instead and avoid the need to introduce a YAML file at all!

In [ ]:
# For reference, this is how the layout is currently retrieved for our configs:
from jdaviz.core.config import read_configuration
read_configuration('specviz')

In [ ]:
# A new prototype config constructor
# Currently missing: How to specify the specific layout of the viewers themselves. Currently only adds them to the same row
class jdaviz_config():
    default_viewer_names = {
        'spectrum1d': {'name': 'Spectrum', 'reference': 'spectrum-viewer'},
        'spectrum2d': {'name': '2D Spectrum viewer', 'reference': 'spectrum-2d-viewer'},
        'image' : {'name': 'Image-viewer', 'reference': 'image-viewer'},
        'table': {'name': 'Table viewer', 'reference': 'table-viewer'},
        'time': {'name': 'Time-viewer', 'reference': 'time-viewer'}
    }

    default_plot_classes = {
        'spectrum1d': 'specviz-profile-viewer',
        'spectrum2d': 'mosviz-profile-2d-viewer',
        'image': 'mosviz-image-viewer',
        'table': 'mosviz-table-viewer',
        'time': 'lcviz-time-viewer'
    }
    
    def __init__(self, config, plugins=None, tools=None, viewers=None):

        self.settings = {'configuration': config,
                         'visible': {'menu_bar': False,
                                     'toolbar': True,
                                     'tray': True,
                                     'tab_headers': False
                                    },
                         'dense_toolbar': False,
                         'context': {'notebook': {'max_height': '600px'}}
                        }
        
        self.viewers = []
        for viewer in viewers:
            self.add_viewer(viewer)         
        
        self.toolbar = []
        for tool in tools:
            self.toolbar.append(tool)

        self.plugins = []
        for plugin in plugins:
            self.plugins.append(plugin)
        
    
    def get_config(self):
        viewer_area = [
                       {'container': 'col',
                        'children' : [
                                      {'container': 'row',
                                       'viewers': self.viewers
                                      }
                                     ]
                       }
                      ]
        return {'settings'   : self.settings,
                'toolbar'    : self.toolbar,
                'tray'       : self.plugins,
                'viewer_area': viewer_area
               }
    
    
    def add_viewer(self, type, name=None, viewer_register=None, reference=None):
        '''
        type: str
            Specifies the type of viewer. Must be one of [spectrum1d, spectrum2d, image, table]
        '''
        if not name:
            name = self.default_viewer_names[type]['name']
        if not viewer_register:
            viewer_register = self.default_plot_classes[type]
        if not reference:
            reference = self.default_viewer_names[type]['reference']
        
        self.viewers.append({'name': name,
                             'plot': viewer_register,
                             'reference': reference
                            })
        
    
    def add_tool(self, tool):
        self.toolbar.append(tool)
    
    
    def add_plugin(self, plugin):
        self.config['tray'].append(plugin)

        
    def register_parser(self, parser):
        raise NotImplementedError
    
    
    def register_viewer(self, viewer, viewer_name):
        raise NotImplementedError
        

In [ ]:
# Here's how you would expect to call it manually
viewers=['image', 'spectrum1d']
tools=['g-subset-tools']
plugins=['g-metadata-viewer', 'g-plot-options']

x = jdaviz_config(config='customviz', viewers=viewers, tools=tools, plugins=plugins)
x.get_config()

In [ ]:
# From within the Viz tool class, here's how simple it would be to request a new layout.
# This does not require any changes to Jdaviz!

from jdaviz.core.helpers import ConfigHelper
import lcviz

class Customviz(ConfigHelper):
    
    _viewers = ['time']
    
    _plugins = ['g-metadata-viewer', 'g-plot-options', 'HelloWorldPlugin']
    
    _tools = ['g-data-tools', 'g-subset-tools']
    
    _default_configuration = jdaviz_config(config='customviz', viewers=_viewers, tools=_tools, plugins=_plugins).get_config()
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._default_time_viewer_reference_name = jdaviz_config.default_viewer_names['time']['reference']
        
    
    def load_data(self, flux, time, data_label):
        '''
        Passes a Spectrum1D to the dummy lcviz parser

        Parameters
        ----------
        data : str, `~specutils.Spectrum1D`, or `~specutils.SpectrumList`
            Spectrum1D, SpectrumList, or path to compatible data file.
        data_label : str
            The Glue data label found in the ``DataCollection``.        
        '''
        data_to_load = Data(x=time.value, flux=flux.value)
        data_to_load.get_component('x').units = str(time.unit)
        data_to_load.get_component('flux').units = str(flux.unit)
        super().load_data(data=data_to_load, parser_reference='lcviz_manual_data_parser',
                          data_label=data_label)       

import lcviz
lcviz.Customviz = Customviz

In [ ]:
from lcviz import Customviz as lcviz
lcviz = lcviz()
lcviz.show()

In [ ]:
import astropy.units as u
from glue.core import Data

time = [0,1,2,3,4,5,6,7,8,9] * u.s
flux = [9,8,7,6,5,4,3,2,1,0] * u.Jy

lcviz.load_data(time=time, flux=flux, data_label="mydata")